# Trial-Averaged Frame Analysis

Average specific frames across trials for left/right before/stim conditions.

**Data structure:**
- 4 CSV files define which frames to average for each condition
- Each CSV: rows = trials, columns = frame positions within trial
- Values are frame indices into the data.bin files

In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mbo_utilities as mbo
from tifffile import imwrite

In [2]:
# Paths
s2p_dir = Path(r"\\rbo-w1\D\W1_DATA\wsnyder\2025-10-16-Females-Shank-Wheel-316902\suite2p")
avg_frames_dir = Path(r"\\rbo-w1\W1_E_USER_DATA\foconnell\lbm\wsnyder")
output_dir = Path().home() / "mbo\data"
output_dir.mkdir(exist_ok=True)

print(f"Suite2p dir: {s2p_dir}")
print(f"CSV dir: {avg_frames_dir}")
print(f"Output dir: {output_dir}")

Suite2p dir: \\rbo-w1\D\W1_DATA\wsnyder\2025-10-16-Females-Shank-Wheel-316902\suite2p
CSV dir: \\rbo-w1\W1_E_USER_DATA\foconnell\lbm\wsnyder
Output dir: C:\Users\RBO\mbo\data


In [3]:
stitched_folders = sorted([p for p in s2p_dir.iterdir() if p.is_dir() and "stitched" in p.name])
for f in stitched_folders:
    print(f"  {f.name}")

  plane01_stitched
  plane02_stitched
  plane03_stitched
  plane04_stitched
  plane05_stitched
  plane06_stitched
  plane07_stitched
  plane08_stitched
  plane09_stitched
  plane10_stitched
  plane11_stitched
  plane12_stitched
  plane13_stitched
  plane14_stitched


In [4]:
# Load all 4 CSV tables
csv_files = {
    "left_before": avg_frames_dir / "Left_Frame_Table_Before.csv",
    "left_stim": avg_frames_dir / "Left_Frame_Table_Stim.csv",
    "right_before": avg_frames_dir / "Right_Frame_Table_Before.csv",
    "right_stim": avg_frames_dir / "Right_Frame_Table_Stim.csv",
}

frame_tables = {}
for name, path in csv_files.items():
    df = pd.read_csv(path)
    frame_tables[name] = df
    print(f"{name}: {df.shape[0]} trials with {df.shape[1]} frames/trial = {df.size} total frames")

left_before: 37 trials with 24 frames/trial = 888 total frames
left_stim: 37 trials with 17 frames/trial = 629 total frames
right_before: 10 trials with 24 frames/trial = 240 total frames
right_stim: 10 trials with 26 frames/trial = 260 total frames


In [5]:
# Preview one of the tables
print("Left Before table (first 5 rows):")
frame_tables["left_before"].head()

Left Before table (first 5 rows):


,Frame 1,Frame 2,Frame 3,Frame 4,Frame 5,Frame 6,Frame 7,Frame 8,Frame 9,Frame 10,...,Frame 15,Frame 16,Frame 17,Frame 18,Frame 19,Frame 20,Frame 21,Frame 22,Frame 23,Frame 24
0,874,875,876,877,878,879,880,881,882,883,...,888,889,890,891,892,893,894,895,896,897
1,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,...,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686
2,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,...,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349
3,5377,5378,5379,5380,5381,5382,5383,5384,5385,5386,...,5391,5392,5393,5394,5395,5396,5397,5398,5399,5400
4,5849,5850,5851,5852,5853,5854,5855,5856,5857,5858,...,5863,5864,5865,5866,5867,5868,5869,5870,5871,5872


In [6]:
# Store results: {condition: {plane_num: trial_averaged_movie}}
# Each movie has shape (17, H, W) - consistent across all conditions
# Before conditions: take LAST 17 frames (leading up to stim)
# Stim conditions: take FIRST 17 frames (after stim onset)

N_FRAMES_OUTPUT = 17  # Consistent output length for all conditions

results = {cond: {} for cond in frame_tables.keys()}

# Process each plane
for folder in stitched_folders:
    bin_path = folder / "data.bin"
    if not bin_path.is_file():
        print(f"Skipping {folder.name} - no data.bin")
        continue

    # Extract plane number from folder name
    plane_name = folder.name
    plane_num = int(''.join(filter(str.isdigit, plane_name.split("plane")[-1])))

    print(f"\nProcessing {plane_name}...")
    data = mbo.imread(bin_path)
    H, W = data.shape[1], data.shape[2]
    print(f"  Data shape: {data.shape}")

    # For each condition
    for condition, df in frame_tables.items():
        n_trials, n_frame_positions = df.shape

        # Determine which frames to use based on condition type
        if "before" in condition:
            # Take LAST 17 frames (leading up to stim)
            frame_cols = list(range(n_frame_positions - N_FRAMES_OUTPUT, n_frame_positions))
        else:  # "stim" conditions
            # Take FIRST 17 frames (after stim onset)
            frame_cols = list(range(N_FRAMES_OUTPUT))

        # Create movie: average across trials for each frame position
        avg_movie = np.zeros((N_FRAMES_OUTPUT, H, W), dtype=np.float32)

        for i, frame_col in enumerate(frame_cols):
            # Get this frame position from all trials
            indices = df.iloc[:, frame_col].astype(int).values
            # Average across trials for this frame position
            avg_movie[i] = data[indices, :, :].mean(axis=0)

        results[condition][plane_num] = avg_movie

        # Save as movie (T, H, W)
        out_path = output_dir / f"{condition}_plane{plane_num}.tif"
        imwrite(out_path, avg_movie)
        print(f"  {condition}: {n_trials} trials × {n_frame_positions} frames -> {out_path.name} shape {avg_movie.shape}")

print("\nDone processing all planes!")


Processing plane01_stitched...
  Data shape: (21880, 594, 458)
  left_before: 37 trials × 24 frames -> left_before_plane1.tif shape (17, 594, 458)
  left_stim: 37 trials × 17 frames -> left_stim_plane1.tif shape (17, 594, 458)
  right_before: 10 trials × 24 frames -> right_before_plane1.tif shape (17, 594, 458)
  right_stim: 10 trials × 26 frames -> right_stim_plane1.tif shape (17, 594, 458)

Processing plane02_stitched...
  Data shape: (21880, 594, 458)
  left_before: 37 trials × 24 frames -> left_before_plane2.tif shape (17, 594, 458)
  left_stim: 37 trials × 17 frames -> left_stim_plane2.tif shape (17, 594, 458)
  right_before: 10 trials × 24 frames -> right_before_plane2.tif shape (17, 594, 458)
  right_stim: 10 trials × 26 frames -> right_stim_plane2.tif shape (17, 594, 458)

Processing plane03_stitched...
  Data shape: (21880, 594, 458)
  left_before: 37 trials × 24 frames -> left_before_plane3.tif shape (17, 594, 458)
  left_stim: 37 trials × 17 frames -> left_stim_plane3.tif s

In [7]:
skip_planes = np.arange(9, 15, 1)
skip_planes

array([ 9, 10, 11, 12, 13, 14])

In [8]:
# Stack all planes into 4D volumes (T, Z, H, W)
# Now each condition has temporal structure preserved

volumes = {}
for condition in results.keys():
    planes = sorted(results[condition].keys())
    # Stack: list of (T, H, W) -> (T, Z, H, W) by stacking along new axis
    plane_movies = [results[condition][p] for i, p in enumerate(planes) if i not in skip_planes]
    # Each movie may have different T, so we need to handle that
    # For now, stack along Z axis: (T, H, W) -> (Z, T, H, W) then transpose to (T, Z, H, W)
    volume = np.stack(plane_movies, axis=0)  # (Z, T, H, W)
    volume = volume.transpose(1, 0, 2, 3)     # (T, Z, H, W)
    volumes[condition] = volume

    volume_path = output_dir / f"{condition}_all_planes.tif"
    imwrite(volume_path, volume.astype(np.float32))
    print(f"Saved 4D volume: {volume_path.name} - shape {volume.shape} (T, Z, H, W)")

Saved 4D volume: left_before_all_planes.tif - shape (17, 9, 594, 458) (T, Z, H, W)
Saved 4D volume: left_stim_all_planes.tif - shape (17, 9, 594, 458) (T, Z, H, W)
Saved 4D volume: right_before_all_planes.tif - shape (17, 9, 594, 458) (T, Z, H, W)
Saved 4D volume: right_stim_all_planes.tif - shape (17, 9, 594, 458) (T, Z, H, W)


In [9]:
# Compute difference images (stim - before)
diff_left = {}
diff_right = {}

for plane_num in results["left_before"].keys():
    diff_left[plane_num] = results["left_stim"][plane_num] - results["left_before"][plane_num]
    diff_right[plane_num] = results["right_stim"][plane_num] - results["right_before"][plane_num]

# Save difference volumes
diff_left_vol = np.stack([diff_left[p] for p in sorted(diff_left.keys())], axis=0)
diff_right_vol = np.stack([diff_right[p] for p in sorted(diff_right.keys())], axis=0)

imwrite(output_dir / "diff_left_stim_minus_before.tif", diff_left_vol.astype(np.float32))
imwrite(output_dir / "diff_right_stim_minus_before.tif", diff_right_vol.astype(np.float32))
print("Saved difference volumes")

Saved difference volumes


In [10]:
# Compute difference images (stim - before)
# Now all conditions have the same shape (17, H, W), so we can directly subtract
diff_left = {}
diff_right = {}

for plane_num in results["left_before"].keys():
    diff_left[plane_num] = results["left_stim"][plane_num] - results["left_before"][plane_num]
    diff_right[plane_num] = results["right_stim"][plane_num] - results["right_before"][plane_num]

# Save difference volumes
diff_left_vol = np.stack([diff_left[p] for p in sorted(diff_left.keys())], axis=0)
diff_right_vol = np.stack([diff_right[p] for p in sorted(diff_right.keys())], axis=0)

imwrite(output_dir / "diff_left_stim_minus_before.tif", diff_left_vol.astype(np.float32))
imwrite(output_dir / "diff_right_stim_minus_before.tif", diff_right_vol.astype(np.float32))
print(f"Saved difference volumes: {diff_left_vol.shape} (Z, T, H, W)")

Saved difference volumes: (14, 17, 594, 458) (Z, T, H, W)


In [11]:
# View in napari (3D)
import napari

viewer = napari.Viewer()

# Add the 4 condition volumes
viewer.add_image(volumes["left_before"], name="Left Before", colormap='gray', scale=(1, 6, 1, 1))
viewer.add_image(volumes["left_stim"], name="Left Stim", colormap='gray', visible=False, scale=(1, 6, 1, 1))
viewer.add_image(volumes["right_before"], name="Right Before", colormap='gray', visible=False, scale=(1, 6, 1, 1))
viewer.add_image(volumes["right_stim"], name="Right Stim", colormap='gray', visible=False, scale=(1, 6, 1, 1))

# Add difference volumes
viewer.add_image(diff_left_vol, name="Left Diff (Stim-Before)", colormap='RdBu_r', visible=False, scale=(1, 6, 1, 1))
viewer.add_image(diff_right_vol, name="Right Diff (Stim-Before)", colormap='RdBu_r', visible=False, scale=(1, 6, 1, 1,))

viewer.show(block=True)

In [12]:
# Load fluorescence traces for heatmaps
import lbm_suite2p_python as lsp

# Select planes to analyze
planes_for_heatmap = [1, 2, 3]  # Change these as needed

# Load traces for each plane
traces_by_plane = {}
for plane_num in planes_for_heatmap:
    folder = s2p_dir / f"plane{plane_num:02d}_stitched"
    if folder.exists():
        F, Fneu, spks, stat, ops, iscell = lsp.load_planar_results(folder)
        # Filter to only "is cell" ROIs
        cell_mask = iscell[:, 0].astype(bool)
        traces_by_plane[plane_num] = {
            'F': F[cell_mask],
            'Fneu': Fneu[cell_mask],
            'spks': spks[cell_mask],
            'stat': [s for s, ic in zip(stat, cell_mask) if ic],
            'n_cells': cell_mask.sum()
        }
        print(f"Plane {plane_num}: {traces_by_plane[plane_num]['n_cells']} cells, {F.shape[1]} frames")

FileNotFoundError: Missing required files in D:\W1_DATA\wsnyder\2025-10-16-Females-Shank-Wheel-316902\suite2p\plane01_stitched: F.npy, Fneu.npy, spks.npy, stat.npy, iscell.npy

In [ ]:
# Load fluorescence traces for heatmaps
# Note: lsp.load_planar_results uses ops["save_path"] which has local D:\ path
# We need to load directly from the UNC path

# Select planes to analyze
planes_for_heatmap = [3, 5, 7]  # Change these as needed

# Load traces for each plane directly from UNC path
traces_by_plane = {}
for plane_num in planes_for_heatmap:
    folder = s2p_dir / f"plane{plane_num:02d}_stitched"
    if folder.exists():
        # Load directly instead of using lsp.load_planar_results
        # (avoids issue with save_path pointing to local D:\ instead of UNC)
        F = np.load(folder / "F.npy")
        Fneu = np.load(folder / "Fneu.npy")
        spks = np.load(folder / "spks.npy")
        stat = np.load(folder / "stat.npy", allow_pickle=True)
        iscell = np.load(folder / "iscell.npy", allow_pickle=True)

        # Filter to only "is cell" ROIs
        cell_mask = iscell[:, 0].astype(bool)
        traces_by_plane[plane_num] = {
            'F': F[cell_mask],
            'Fneu': Fneu[cell_mask],
            'spks': spks[cell_mask],
            'stat': [s for s, ic in zip(stat, cell_mask) if ic],
            'n_cells': cell_mask.sum()
        }
        print(f"Plane {plane_num}: {traces_by_plane[plane_num]['n_cells']} cells, {F.shape[1]} frames")

In [20]:
# Create heatmaps for left trials - activity around trial onset
window_before = 24  # frames before trial start
window_after = 30   # frames after trial start

fig, axes = plt.subplots(len(planes_for_heatmap), 2, figsize=(14, 4 * len(planes_for_heatmap)))
fig.suptitle("Neural Activity Around Left Trial Onset", fontsize=14, y=1.02)

for row, plane_num in enumerate(planes_for_heatmap):
    if plane_num not in traces_by_plane:
        continue

    F = traces_by_plane[plane_num]['F']

    # Extract traces around left trials (before period)
    left_traces, onset_idx = extract_trial_traces(
        F, frame_tables["left_before"],
        window_before=window_before, window_after=window_after
    )

    # Average across trials: (n_cells, n_timepoints)
    avg_traces = left_traces.mean(axis=0)

    # Z-score normalize each cell
    mean_per_cell = avg_traces.mean(axis=1, keepdims=True)
    std_per_cell = avg_traces.std(axis=1, keepdims=True)
    std_per_cell[std_per_cell == 0] = 1  # avoid divide by zero
    zscore_traces = (avg_traces - mean_per_cell) / std_per_cell

    # Sort cells by peak time
    peak_times = np.argmax(zscore_traces, axis=1)
    sort_idx = np.argsort(peak_times)

    # Time axis (relative to trial onset)
    time_axis = np.arange(-window_before, window_after)

    # Plot heatmap
    ax = axes[row, 0] if len(planes_for_heatmap) > 1 else axes[0]
    im = ax.imshow(zscore_traces[sort_idx], aspect='auto', cmap='viridis',
                   extent=[time_axis[0], time_axis[-1], len(sort_idx), 0])
    ax.axvline(0, color='white', linestyle='--', linewidth=1, label='Trial onset')
    ax.set_xlabel("Frames from trial onset")
    ax.set_ylabel("Cells (sorted by peak)")
    ax.set_title(f"Plane {plane_num} - Left Trials (n={left_traces.shape[0]})")
    plt.colorbar(im, ax=ax, label='Z-score')

    # Plot average trace across all cells
    ax2 = axes[row, 1] if len(planes_for_heatmap) > 1 else axes[1]
    mean_trace = zscore_traces.mean(axis=0)
    sem_trace = zscore_traces.std(axis=0) / np.sqrt(len(zscore_traces))
    ax2.fill_between(time_axis, mean_trace - sem_trace, mean_trace + sem_trace, alpha=0.3)
    ax2.plot(time_axis, mean_trace, 'b-', linewidth=2)
    ax2.axvline(0, color='red', linestyle='--', linewidth=1, label='Trial onset')
    ax2.set_xlabel("Frames from trial onset")
    ax2.set_ylabel("Mean Z-score ± SEM")
    ax2.set_title(f"Plane {plane_num} - Population Average")
    ax2.legend()

plt.tight_layout()
plt.show()

NameError: name 'planes_for_heatmap' is not defined

In [21]:
# Compare Left vs Right trials for a single plane
compare_plane = 5

if compare_plane in traces_by_plane:
    F = traces_by_plane[compare_plane]['F']

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f"Plane {compare_plane}: Left vs Right Trial Comparison", fontsize=14)

    conditions = [
        ("left_before", "Left Before", 'Blues'),
        ("left_stim", "Left Stim", 'Reds'),
        ("right_before", "Right Before", 'Greens'),
        ("right_stim", "Right Stim", 'Oranges'),
    ]

    for ax, (cond_key, title, cmap) in zip(axes.flat, conditions):
        traces, onset_idx = extract_trial_traces(
            F, frame_tables[cond_key],
            window_before=window_before, window_after=window_after
        )

        # Average across trials
        avg_traces = traces.mean(axis=0)

        # Z-score
        mean_per_cell = avg_traces.mean(axis=1, keepdims=True)
        std_per_cell = avg_traces.std(axis=1, keepdims=True)
        std_per_cell[std_per_cell == 0] = 1
        zscore_traces = (avg_traces - mean_per_cell) / std_per_cell

        # Sort by peak
        peak_times = np.argmax(zscore_traces, axis=1)
        sort_idx = np.argsort(peak_times)

        time_axis = np.arange(-window_before, window_after)

        im = ax.imshow(zscore_traces[sort_idx], aspect='auto', cmap=cmap,
                       extent=[time_axis[0], time_axis[-1], len(sort_idx), 0])
        ax.axvline(0, color='black', linestyle='--', linewidth=2)
        ax.set_xlabel("Frames from trial onset")
        ax.set_ylabel("Cells (sorted)")
        ax.set_title(f"{title} (n={traces.shape[0]} trials)")
        plt.colorbar(im, ax=ax, label='Z-score')

    plt.tight_layout()
    plt.show()

NameError: name 'traces_by_plane' is not defined

In [ ]:
# Population average comparison: Left vs Right
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle(f"Plane {compare_plane}: Population Average Comparison", fontsize=14)

colors = {'left_before': 'blue', 'left_stim': 'red',
          'right_before': 'green', 'right_stim': 'orange'}

F = traces_by_plane[compare_plane]['F']
time_axis = np.arange(-window_before, window_after)

# Left panel: Before vs Stim for Left trials
ax = axes[0]
for cond_key in ['left_before', 'left_stim']:
    traces, _ = extract_trial_traces(F, frame_tables[cond_key],
                                      window_before=window_before, window_after=window_after)
    avg = traces.mean(axis=0)  # (n_cells, n_time)

    # Z-score
    mean_per_cell = avg.mean(axis=1, keepdims=True)
    std_per_cell = avg.std(axis=1, keepdims=True)
    std_per_cell[std_per_cell == 0] = 1
    zscore = (avg - mean_per_cell) / std_per_cell

    pop_mean = zscore.mean(axis=0)
    pop_sem = zscore.std(axis=0) / np.sqrt(len(zscore))

    label = cond_key.replace('_', ' ').title()
    ax.fill_between(time_axis, pop_mean - pop_sem, pop_mean + pop_sem,
                    alpha=0.2, color=colors[cond_key])
    ax.plot(time_axis, pop_mean, color=colors[cond_key], linewidth=2, label=label)

ax.axvline(0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel("Frames from trial onset")
ax.set_ylabel("Mean Z-score ± SEM")
ax.set_title("Left Trials: Before vs Stim")
ax.legend()

# Right panel: Before vs Stim for Right trials
ax = axes[1]
for cond_key in ['right_before', 'right_stim']:
    traces, _ = extract_trial_traces(F, frame_tables[cond_key],
                                      window_before=window_before, window_after=window_after)
    avg = traces.mean(axis=0)

    mean_per_cell = avg.mean(axis=1, keepdims=True)
    std_per_cell = avg.std(axis=1, keepdims=True)
    std_per_cell[std_per_cell == 0] = 1
    zscore = (avg - mean_per_cell) / std_per_cell

    pop_mean = zscore.mean(axis=0)
    pop_sem = zscore.std(axis=0) / np.sqrt(len(zscore))

    label = cond_key.replace('_', ' ').title()
    ax.fill_between(time_axis, pop_mean - pop_sem, pop_mean + pop_sem,
                    alpha=0.2, color=colors[cond_key])
    ax.plot(time_axis, pop_mean, color=colors[cond_key], linewidth=2, label=label)

ax.axvline(0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel("Frames from trial onset")
ax.set_ylabel("Mean Z-score ± SEM")
ax.set_title("Right Trials: Before vs Stim")
ax.legend()

plt.tight_layout()
plt.show()

## Output Summary

Files saved to output directory:
- `left_before_plane{N}.tif` - Individual plane averages
- `left_stim_plane{N}.tif`
- `right_before_plane{N}.tif`
- `right_stim_plane{N}.tif`
- `left_before_all_planes.tif` - 3D volume (Z × H × W)
- `left_stim_all_planes.tif`
- `right_before_all_planes.tif`
- `right_stim_all_planes.tif`
- `diff_left_stim_minus_before.tif` - Difference volume
- `diff_right_stim_minus_before.tif`

## Heatmap Analysis

The heatmaps show:
- **Rows**: Individual cells, sorted by peak response time
- **Columns**: Time relative to trial onset (frame 0 = first frame of trial)
- **Color**: Z-scored fluorescence activity

Population averages show mean ± SEM across all cells for each condition.